In [3]:
from src.utility import get_root
import pandas as pd
import os
from typing import Tuple, List, Optional

In [4]:
def estimate_flops(model: str, input_size: Tuple[int, int], training_strategy: str, sample_count: int, estimated_epochs: int) -> float:
    path_to_flops = os.path.join(get_root(), "data", "model_flops", "model_flops.xlsx")
    flops_df = pd.read_excel(path_to_flops)

    model_info = flops_df[flops_df["Model"] == model]

    if model_info.empty:
        raise ValueError(f"Model {model} not found in the flops database")
    
    model_type = model_info['Type'].iloc[0]
    original_input_size = model_info['Input Size'].iloc[0].split()[0]

    if model_type == 'Vision':
        width, height = map(int, original_input_size.split('x'))
        scaling = (input_size[0] * input_size[1]) / (width * height)
    else:
        scaling = input_size[0] / int(original_input_size)

    if training_strategy in ["Fine-tuning the whole model", "Full Training"]:
        return estimated_epochs * sample_count * model_info['FLOPs'].iloc[0] * scaling * 3
    elif training_strategy == "Last Layer Learning":
        return estimated_epochs * sample_count * 2 * model_info['Last Layer FLOPs'].iloc[0] + model_info['FLOPs'].iloc[0] * scaling
    else:
        raise ValueError(f"Unsupported training strategy: {training_strategy}")


In [5]:
def estimate_time(flops: float, gpu: str, training_strategy: str, tflops: str) -> float:
    path_to_gpu = os.path.join(get_root(), "data", "gpus.csv")
    gpu_df = pd.read_csv(path_to_gpu)
    gpu_info = gpu_df[gpu_df["name"] == gpu]

    if gpu_info.empty:
        raise ValueError(f"GPU {gpu} not found in the flops database")
    
    return flops / gpu_info[tflops].iloc[0] / 1e+12 if training_strategy in ["Full Training", "Fine-tuning the whole model"] else flops / gpu_info[tflops].iloc[0] / 1e+12 / 3

In [6]:
def calculate_kwh_consumption(gpu_name, time_seconds):
    path_to_gpu = os.path.join(get_root(), "data", "gpus.csv")
    gpu_df = pd.read_csv(path_to_gpu)

    tdp_watts = gpu_df.loc[gpu_df['name'] == gpu_name, 'tdp_watts'].values[0]
    
    # Convert TDP from watts to kilowatts
    tdp_kw = tdp_watts / 1000
    
    # Convert time from seconds to hours
    time_hours = time_seconds / 3600
    
    # Calculate the energy consumption in kWh
    energy_consumption_kwh = tdp_kw * time_hours
    
    return energy_consumption_kwh


In [7]:
a = estimate_flops("fasterrcnn_resnet50_fpn", (256, 256), "Fine-tuning the whole model", 1000000, 40)
print(a)

2.106196446814041e+19


In [8]:
b = estimate_time(a, "RTX 3080 TI", "Fine-tuning the whole model", "TFLOPS16")

In [9]:
c = calculate_kwh_consumption("RTX 3080 TI", b)
c

60.049589148331236

In [49]:
def normalize_data(data):
    return (data - data.min()) / (data.max() - data.min())



def get_tflops_value(perf_data, tflops_type):
    if tflops_type in perf_data:
        return perf_data[tflops_type]
    elif 'TFLOPS32' in perf_data:
        return perf_data['TFLOPS32']
    elif 'TFLOPS16' in perf_data:
        return perf_data['TFLOPS16']
    else:
        raise ValueError("No valid TFLOPS column found for the GPU.")

def recommend_gpu(scores, tflops_type):
    pricing_df = pd.read_excel(os.path.join(get_root(), 'data', 'pricing', 'GCP gpus pricing.xlsx'))
    pricing_df.columns = ['Region', 'GPU', 'Price']
    gpu_prices = pricing_df.groupby('GPU')['Price'].mean().reset_index()
    gpu_prices['Normalized_Price'] = normalize_data(gpu_prices['Price'])
    
    performance_df = pd.read_csv(os.path.join(get_root(), 'data', 'gpus.csv'))
    
    manual_map = {
        'T4': 'T4',
        'V100': 'Tesla V100-PCIE-16GB',
        'P100': 'Tesla P100',
        'K80': 'Tesla K80',
    }
    
    gpu_prices['Mapped_GPU'] = gpu_prices['GPU'].apply(lambda x: manual_map.get(x, x))
    performance_data = {row['name']: row for _, row in performance_df.iterrows()}
    
    merged_data = []
    for _, row in gpu_prices.iterrows():
        pricing_gpu = row['Mapped_GPU']
        if pricing_gpu in performance_data:
            perf_data = performance_data[pricing_gpu]
            print(perf_data)
            tflops_value = get_tflops_value(perf_data, tflops_type)
            print(tflops_value)
            tflops_per_watt = tflops_value / perf_data['tdp_watts']
            merged_data.append({
                'GPU': row['GPU'],
                'Mapped_GPU': pricing_gpu,
                'Price': row['Price'],
                'Normalized_Price': row['Normalized_Price'],
                'TDP_Watts': perf_data['tdp_watts'],
                tflops_type: tflops_value,
                'TFLOPS_per_Watt': tflops_per_watt
            })
    
    merged_df = pd.DataFrame(merged_data)
    
    merged_df['Normalized_TFLOPS'] = normalize_data(merged_df[tflops_type])
    merged_df['Normalized_TDP'] = normalize_data(merged_df['TDP_Watts'])
    merged_df['Normalized_TFLOPS_per_Watt'] = normalize_data(merged_df['TFLOPS_per_Watt'])

    merged_df['Price_Score'] = (1 - merged_df['Normalized_Price']) * scores['price']
    merged_df['TFLOPS_Score'] = merged_df['Normalized_TFLOPS'] * scores['time']
    merged_df['TDP_Score'] = merged_df['Normalized_TFLOPS_per_Watt'] * scores['co2']

    merged_df['Total_Score'] = merged_df['Price_Score'] + merged_df['TFLOPS_Score'] + merged_df['TDP_Score']
    
    best_gpu = merged_df.sort_values('Total_Score', ascending=False).iloc[0]
    
    return best_gpu['Mapped_GPU']


In [50]:
import random
if __name__ == "__main__":
    tflops_type = 'TFLOPS16'  # or 'TFLOPS16'
    
    num_samples = 1
    results = []

    for _ in range(num_samples):
        weights = {
            'price': random.uniform(0, 1),
            'time': random.uniform(0, 1),
            'co2': random.uniform(0, 1)
        }
        total = sum(weights.values())
        weights = {k: v / total for k, v in weights.items()}  # Normalize to sum to 1
        best_gpu = recommend_gpu(weights, tflops_type)
        results.append((weights, best_gpu))
    
    # Print or save the results
    for weights, gpu in results:
        print(f"With weights {weights}, the recommended GPU is: {gpu}")


name                Tesla K80
type                      gpu
tdp_watts                 300
TFLOPS32                4.113
TFLOPS16                  NaN
GFLOPS32/W              13.71
GFLOPS16/W                NaN
memory                   12.0
source        techpowerup.com
Name: 15, dtype: object
##################################################
name               Tesla P100
type                      gpu
tdp_watts                 250
TFLOPS32                9.526
TFLOPS16                19.05
GFLOPS32/W               38.1
GFLOPS16/W               76.2
memory                   16.0
source        techpowerup.com
Name: 17, dtype: object
##################################################
name                                                         T4
type                                                        gpu
tdp_watts                                                    70
TFLOPS32                                                  8.141
TFLOPS16                                              